In [126]:
# importing sufficient libraries for scraping 
import requests
from bs4 import BeautifulSoup

In [127]:
# requesting the page source and checking the status code of targeted website
def get_page(category = None, page = 1):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    page = requests.get(f'https://priceoye.pk/{category}?page={page}', headers=headers)
    return (page)

In [128]:
page = get_page()
page.status_code

200

In [129]:
# content of the page
#page.text

In [130]:
page = get_page()
soup = BeautifulSoup(page.text, 'html.parser')
soup.find('title')

<title>Find the Latest Mobile Prices in Pakistan | PriceOye</title>

# Products Links
Getting links of all the products having multiple pages also

In [131]:
# getting the list of all the elemnets containg the product
product_list = soup.find_all('div', class_= 'productBox b-productBox' )

In [132]:
len(product_list)

35

In [133]:
#getting the links of all the products from the the product element list
product_links = []

for item in product_list:
    for link in item.find_all('a', href = True):
        product_links.append(link['href'])

In [134]:
print(product_links[0])
print(len(product_links))

https://priceoye.pk/mobiles/infinix/infinix-hot-11-play
35


#### Product Link Function
In this function we have to give 2 arguments 
* category name
* total pages the category have

In [135]:
def get_products_links(category = 'mobiles', total_pages = 2):
    product_links = []
    for page_no in range(1,total_pages):
        page = get_page(category, page_no)

        soup = BeautifulSoup(page.text, 'html.parser')

        product_list = soup.find_all('div', class_= 'productBox b-productBox' )

        for item in product_list:
            for link in item.find_all('a', href = True):
                product_links.append(link['href'])
    
    return(product_links)


In [136]:
product_links = get_products_links('tablets')

In [137]:
print(product_links[0])
print(len(product_links))

https://priceoye.pk/tablets/samsung/samsung-tab-a7-lite-8-7-inches-t220
30


# Product Details
Getting details of all the products through the extracted links

In [138]:
test_link = 'https://priceoye.pk/tablets/samsung/samsung-tab-a7-lite-8-7-inches-t220'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
r = requests.get(test_link, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [139]:
# getting title
try:
    title = soup.find('h2', class_ = 'h1').text.strip()
except:
    title = None
print(title)

Samsung Galaxy Tab A7 Lite 8.7 inches (T220)


In [140]:
# getting total number of ratings and average ratings
try:
    num_of_ratings = soup.find('div', class_ = 'h5 semi-bold rating-count').text.strip()
except:
    num_of_ratings = None
try:
    average_rating = soup.find('div', class_ = 'h4 semi-bold rating-points').text.strip()
except:
    average_rating = None
print(num_of_ratings)
print(average_rating)

20 Ratings
4.6


In [141]:
# getting price and old price
try:
    price = soup.find('span', class_ = 'summary-price text-black price-size-lg').text.strip()
except:
    price = None
try:
    old_price = soup.find('span', class_ = 'summary-price line-through').text.strip()
except:
    old_price = None
print('new price', price)
print('old price', old_price)

new price Rs.26,999
old price Rs. 30,000


In [142]:
# getting stock availability
try:
    avaiability = soup.find('span', class_ = 'summary-price text-black').text.strip()
except:
    avaiability = None
print(avaiability)

In Stock


In [143]:
try:
    category = []
    categories = soup.find('div', class_ = 'breadcrumb')
    for x in categories:
        if len(x.text) > 1:
            category.append(x.text)
except:
    category = None
print(category)

['Tablets', 'Samsung', 'Samsung Tab A7 Lite 8 7 Inches T220']


In [146]:
def get_product_details(link):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    page = requests.get(link, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')


    # getting title
    try:
        title = soup.find('h2', class_ = 'h1').text.strip()
    except:
        title = None

    # getting number of ratings
    try:
        num_of_ratings = soup.find('div', class_ = 'h5 semi-bold rating-count').text.strip()
    except:
        num_of_ratings = None
    # getting average rating
    try:
        average_rating = soup.find('div', class_ = 'h4 semi-bold rating-points').text.strip()
    except:
        average_rating = None
    
    # getting price 
    try:
        price = soup.find('span', class_ = 'summary-price text-black price-size-lg').text.strip()
    except:
        price = None
    # getting old price
    try:
        old_price = soup.find('span', class_ = 'summary-price line-through').text.strip()
    except:
        old_price = None

    # getting stock availability
    try:
        avaiability = soup.find('span', class_ = 'summary-price text-black').text.strip()
    except:
        avaiability = None

    #getting categories
    try:
        category = []
        categories = soup.find('div', class_ = 'breadcrumb')
        for x in categories:
            if len(x.text) > 1:
                category.append(x.text)
            break
    except:
        category = None

    detail = {
        'Name': title,
        'Num_of_ratings': num_of_ratings,
        'Average_ratings': average_rating,
        'Price': price,
        'old_price': old_price,
        'Availability': avaiability,
        'Category': category,
        'Link': link,

    }

    return(detail)

In [ ]:
cats = {'mobiles':18, 'tab':2, 'lap':2}

# Testing Function

In [147]:
all_links = []
for key, value in cats:
    product_links = get_products_links(category=key, total_pages=value)
    for links in product_links:
        all_links.append(links)

# product_links = get_products_links(category= )
product_details = []
for link in all_links:
    detail = get_product_details(link)
    product_details.append(detail)

In [148]:
product_details

[{'Name': 'Samsung Galaxy Tab A7 Lite 8.7 inches (T220)',
  'Num_of_ratings': '20 Ratings',
  'Average_ratings': '4.6',
  'Price': 'Rs.26,999',
  'old_price': 'Rs. 30,000',
  'Availability': 'In Stock',
  'Category': ['Tablets', 'Samsung', 'Samsung Tab A7 Lite 8 7 Inches T220'],
  'Link': 'https://priceoye.pk/tablets/samsung/samsung-tab-a7-lite-8-7-inches-t220'},
 {'Name': 'Xiaomi Pad 5',
  'Num_of_ratings': '14 Ratings',
  'Average_ratings': '4.9',
  'Price': 'Rs.67,499',
  'old_price': 'Rs. 70,999',
  'Availability': 'In Stock',
  'Category': ['Tablets', 'Xiaomi', 'Xiaomi Pad 5'],
  'Link': 'https://priceoye.pk/tablets/xiaomi/xiaomi-pad-5'},
 {'Name': 'Dany Signature S8',
  'Num_of_ratings': '0 Ratings',
  'Average_ratings': '',
  'Price': 'Rs.22,599',
  'old_price': 'Rs. 25,900',
  'Availability': 'In Stock',
  'Category': ['Tablets', 'Dany', 'Dany Signature S8'],
  'Link': 'https://priceoye.pk/tablets/dany/dany-signature-s8'},
 {'Name': 'Samsung Galaxy Tab A7 Lite 8.7 inches (T225N

In [149]:
import pandas as pd
df = pd.DataFrame(product_details)

In [150]:
df

,Name,Num_of_ratings,Average_ratings,Price,old_price,Availability,Category,Link
0,Samsung Galaxy Tab A7 Lite 8.7 inches (T220),20 Ratings,4.6,"Rs.26,999","Rs. 30,000",In Stock,"[Tablets, Samsung, Samsung Tab A7 Lite 8 7 Inc...",https://priceoye.pk/tablets/samsung/samsung-ta...
1,Xiaomi Pad 5,14 Ratings,4.9,"Rs.67,499","Rs. 70,999",In Stock,"[Tablets, Xiaomi, Xiaomi Pad 5]",https://priceoye.pk/tablets/xiaomi/xiaomi-pad-5
2,Dany Signature S8,0 Ratings,,"Rs.22,599","Rs. 25,900",In Stock,"[Tablets, Dany, Dany Signature S8]",https://priceoye.pk/tablets/dany/dany-signatur...
3,Samsung Galaxy Tab A7 Lite 8.7 inches (T225N),18 Ratings,4.7,"Rs.28,999","Rs. 34,000",In Stock,"[Tablets, Samsung, Samsung Tab A7]",https://priceoye.pk/tablets/samsung/samsung-ta...
4,QTab Y1,1 Ratings,5.0,"Rs.11,299","Rs. 11,900",In Stock,"[Tablets, Qmobile, Qtab Y1]",https://priceoye.pk/tablets/qmobile/qtab-y1
5,Samsung Galaxy Tab S6 Lite 10.4 inches (P610),0 Ratings,,"Rs.61,999","Rs. 70,000",In Stock,"[Tablets, Samsung, Samsung Galaxy Tab S6 Lite ...",https://priceoye.pk/tablets/samsung/samsung-ga...
6,Nokia Tab T20,0 Ratings,,"Rs.36,299","Rs. 38,000",In Stock,"[Tablets, Nokia, Nokia Tab T20]",https://priceoye.pk/tablets/nokia/nokia-tab-t20
7,Apple iPad Air 5,0 Ratings,,"Rs.121,999","Rs. 124,500",In Stock,"[Tablets, Apple, Apple Ipad Air 5]",https://priceoye.pk/tablets/apple/apple-ipad-a...
8,Dany Monster 4G LTE,0 Ratings,,"Rs.14,599","Rs. 16,900",In Stock,"[Tablets, Dany, Dany Monster 4g Lte]",https://priceoye.pk/tablets/dany/dany-monster-...
9,Samsung Galaxy Tab A7 10.4 inches (T505N),0 Ratings,,"Rs.46,499","Rs. 50,000",In Stock,"[Tablets, Samsung, Samsung Tab A7 10 4 Inches ...",https://priceoye.pk/tablets/samsung/samsung-ta...
